# 《按图索骥学机器学习》-《A05特征处理之标准化》

### 这是《按图索骥学机器学习》-《A05特征处理之标准化》的讲义
这门课程之所以叫按图索骥，是因为学习资料都放到了思维导图当中，大家可以根据自己的情况，选择合适的学习路径，自主学习

![avatar](pic/swnt.png)

导图和有关学习资料都放在了github(git.code946.com)上，并且在不断迭代和更新中

In [37]:
from sklearn.datasets import load_boston

lb= load_boston()

print(lb["DESCR"])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [38]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(lb.data,lb.target,test_size=0.25)

查看训练集和测试集的平均值，标准差是否一致

In [39]:
import numpy as np

print(np.std(x_train))
print(np.std(x_test))
print(np.std(lb.data))

print(np.mean(x_train))
print(np.mean(x_test))
print(np.mean(lb.data))

147.56275863356814
137.69034899997743
145.1555388220164
70.92853130099452
67.52373198061781
70.07396704469443


先做数据分割，再分别对训练集和测试集进行标准化
原因：如果你先对整体数据进行标准化，再分割成训练集和测试集，会导致训练集中包含了测试集的一些信息
    影响测试集对模型评价的准确度
    比方说：测试过程就是高考，测试集数据就是高考题
         学校的模拟题相当于训练集数据，如果训练集中包含了测试集的信息
         相当于模拟题中有高考题的出现，这回影响到最终的成绩

fit_transform 与 transform区别

fit_transform：两个步骤
    fit : 计算中间值：求平均值和标准差，将结果保存到StandardScaler对象中
    transform ：计算标砖化，得到标准化后的最终结果

测试集中的特征值计算标准化的时候，使用的是训练集中的平均值和标准差
理想中的测试集中的数据分布和训练集的数据分布式一致的，那么平均值和标准差也是差不多的

In [40]:
from sklearn.preprocessing import StandardScaler

std_x = StandardScaler()
x_train = std_x.fit_transform(x_train)
x_test = std_x.transform(x_test)

std_y = StandardScaler()
y_train = std_y.fit_transform(y_train.reshape(-1,1))
y_test = std_y.transform(y_test.reshape(-1,1))

print(x_train[:10,0])
print(x_train[:10,9])
print(y_train[:10,0])

[ 0.54999749 -0.32886307 -0.41468882 -0.42509795 -0.41820031 -0.42386026
 -0.2779778   0.47209401 -0.38403796 -0.42348634]
[ 1.42333897  0.1023422  -0.78024515 -1.13789493 -0.82639351 -0.71102262
 -0.09378832  1.42333897 -0.64756862 -0.71102262]
[-1.03194466 -0.73674568  0.07505153  0.6127354   1.69864595  0.71816361
  2.88998472 -0.92651645  0.67599232  0.06450871]


class sklearn.linear_model.SGDRegressor(loss=’squared_loss’, penalty=’l2’, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate=’invscaling’, eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False)

learning_rate=’invscaling’: sk-learn内置的一套参数取值方案
eta0代表的是学习率

梯度下降结束运行的条件：
1、 max_iter:代表的是最大的迭代次数

2、当前的损失值 - 上一次损失值 < epsilon

In [41]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor()
sgd.fit(x_train,y_train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [42]:
y_predict = std_y.inverse_transform(sgd.predict(x_test))
# 输出预测结果时，要对数值进行标准化的逆运算 

print("预测价格：%f" % y_predict[0])

预测价格：27.824319


In [43]:
from sklearn.metrics import mean_squared_error

print("均方误差：%f" % mean_squared_error(std_y.inverse_transform(y_test),y_predict))

均方误差：15.822986
